In [1]:
import torch
import time
from data import train_data_prepare
from train import train
from test import test, test_data_prepare
from model import Net
from model_baseline import BaselineNet
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import pandas as pd
import itertools
import liwc
import numpy as np

In [2]:
from tika import parser

raw = parser.from_file('LIWC2007 dictionary poster.pdf')
#print(raw['content'])

content = raw['content']
rows = content.split("\n")


In [3]:
firstrow_idx = 59
lastrow_idx = 239
columnrow_idx = 52

liwc_categories = rows[columnrow_idx].split(" ")
liwc_categories

['Funct',
 'Pronoun',
 'Ppron',
 'I',
 'We',
 'You',
 'SheHe',
 'They',
 'Ipron',
 'Article',
 'Verbs',
 'AuxVb',
 'Past',
 'Present',
 'Future',
 'Adverbs',
 'Prep',
 'Conj',
 'Negate',
 'Quant',
 'Numbers',
 'Swear',
 'Social',
 'Family',
 'Friends',
 'Humans',
 'Affect',
 'Posemo',
 'Negemo',
 'Anx',
 'Anger',
 'Sad',
 'Insight',
 'Cause',
 'Discrep',
 'Tentat',
 'Certain',
 'Inhib',
 'Incl',
 'Excl',
 'Percept',
 'See',
 'Hear',
 'Feel',
 'Bio',
 'Body',
 'Health',
 'Sexual',
 'Ingest',
 'Relativ',
 'Motion',
 'Space',
 'Time',
 'Work',
 'Achiev',
 'Leisure',
 'Home',
 'Money',
 'Relig',
 'Death',
 'Assent',
 'Nonflu',
 'Filler']

In [4]:

textrows = rows[firstrow_idx:lastrow_idx+1]
textrows_len = len(textrows)

In [5]:
textrows[-1].split(" ")

['']

In [6]:
# find all empty lines

empty_idx = []
for j in range(textrows_len):
    
    size = len(textrows[j].split(" "))
    if size < 2 and textrows[j].split(" ")[0] == '':
        print("len less than 10 at j = " + str(j) + " with size " + str(size))
        print("")
        empty_idx.append(j)
        
textrows_clean = np.delete(textrows, empty_idx).tolist()
textrows_clean_len = len(textrows_clean)

len less than 10 at j = 14 with size 1

len less than 10 at j = 26 with size 1

len less than 10 at j = 41 with size 1

len less than 10 at j = 67 with size 1

len less than 10 at j = 82 with size 1

len less than 10 at j = 108 with size 1

len less than 10 at j = 123 with size 1

len less than 10 at j = 162 with size 1

len less than 10 at j = 165 with size 1

len less than 10 at j = 176 with size 1

len less than 10 at j = 178 with size 1

len less than 10 at j = 179 with size 1

len less than 10 at j = 180 with size 1



In [7]:
toolong_idx = []

textrows_clean2x = textrows_clean.copy()

basesize = len(textrows_clean[0].split(" "))
oldsize = basesize

for j in range(textrows_clean_len):
    
    currsize = len(textrows_clean[j].split(" "))
    
    if currsize > oldsize*1.1:
        print("len longer than max at j = " + str(j) + " with size " + str(size))
        print("")
        toolong_idx.append(j)
    oldsize = currsize

len longer than max at j = 2 with size 1

len longer than max at j = 5 with size 1

len longer than max at j = 8 with size 1

len longer than max at j = 11 with size 1

len longer than max at j = 14 with size 1

len longer than max at j = 16 with size 1

len longer than max at j = 19 with size 1

len longer than max at j = 22 with size 1

len longer than max at j = 25 with size 1

len longer than max at j = 30 with size 1

len longer than max at j = 33 with size 1

len longer than max at j = 36 with size 1

len longer than max at j = 39 with size 1

len longer than max at j = 41 with size 1

len longer than max at j = 44 with size 1

len longer than max at j = 47 with size 1

len longer than max at j = 50 with size 1

len longer than max at j = 55 with size 1

len longer than max at j = 58 with size 1

len longer than max at j = 61 with size 1

len longer than max at j = 64 with size 1

len longer than max at j = 69 with size 1

len longer than max at j = 72 with size 1

len longer tha

In [8]:
len(toolong_idx)

50

In [9]:
curridx = 0
currlist = textrows_clean[toolong_idx[curridx]].split(" ")[:105-1]
currlist[-1] = "imean"
currlist
textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

In [10]:
curridx = 1
currlist = textrows_clean[toolong_idx[curridx]].split(" ")
currlist = currlist[:103]+['rr*']
currlist
textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

In [11]:
curridx = 2
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:102]+['youknow']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)
print(len(currlist))

208
103


In [12]:
curridx = 3
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:100]+['haha*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))


201
101


In [13]:
curridx = 4
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[49:]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))


148
99


In [14]:
curridx = 5
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:97]+["mm*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))



136
98


In [15]:
curridx = 6
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))


currlist = currlist[:95]+['okey*']
                          
textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))



197
96


In [16]:
curridx = 7
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:94] + ["uhuh"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

189
95


In [17]:
curridx = 8
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))


currlist = currlist[173:]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

268
95


In [18]:
curridx = 9
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:92] + ['genocid*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

188
93


In [19]:
curridx = 10
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:92]+['grief']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

191
93


In [20]:
curridx = 11
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:90]+['holocaust*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

181
91


In [21]:
curridx = 12
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist=["away"]+currlist[156:]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

246
91


In [22]:
curridx = 13
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:90]+['lynch*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

135
91


In [23]:
curridx = 14
currlist = textrows_clean[toolong_idx[curridx]].split(" ")


print(len(currlist))
currlist = currlist[:90]+["mausoleum"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

185
91


In [24]:
curridx = 15
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))


currlist = currlist[:88] + ["mortician"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

182
89


In [25]:
curridx = 16
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:88] + ['obit*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

177
89


In [26]:
curridx = 17
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:86] + ['plague*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

113
87


In [27]:
curridx = 18
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:85]+ ["suicid"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

176
86


In [28]:
curridx = 19
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:83]+["war"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

167
84


In [29]:
curridx = 20
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = ["can"]+currlist[146:]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

228
83


In [30]:
curridx = 21
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:80]+["temple"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

165
81


In [31]:
curridx = 22
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist  = currlist[:77]+["torah"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

159
78


In [32]:
curridx = 23
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:77] + ["worship*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

155
78


In [33]:
curridx = 24
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = ["cuz"] + currlist[139:]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

214
76


In [34]:
curridx = 25
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:73] + ['wager*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

108
74


In [35]:
curridx = 26
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:73] + ["worth"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

151
74


In [36]:
curridx = 27
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:70]+['IRS']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

142
71


In [37]:
curridx = 28
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:67] + ["toaster*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))



135
68


In [38]:
curridx = 29
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:60]+['travel*']

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

76
61


In [39]:
curridx = 30
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:58]+["unwind"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

118
59


In [40]:
curridx = 31
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:57]+["video*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

115
58


In [41]:
curridx = 32
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = ["entire*"] + currlist[107:]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

163
57


In [42]:
curridx = 33
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:56] + ["weights"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


print(len(currlist))

77
57


In [43]:
curridx = 34
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:55] + ["wine"] 

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

114
56


In [44]:
curridx = 35
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:48]+["xbox"]


print(len(currlist))


textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)


98
49


In [45]:
curridx = 36
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:47]+["margarita*"]


print(len(currlist))


textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

95
48


In [46]:
curridx = 37
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = ["extra"] + currlist[80:]
print(len(currlist))


textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

127
48


In [47]:
curridx = 38
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:43]+["youth*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))


60
44


In [48]:
curridx = 39
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:41]+["receiv*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

84
42


In [49]:
curridx = 40
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:37]+["replacing"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

76
38


In [50]:
curridx = 41
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))
currlist = currlist[:28]+["rides"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

57
29


In [51]:
curridx = 42
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:29]+["run"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

62
30


In [52]:
curridx = 43
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:26]+["runs"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

53
27


In [53]:
curridx = 44
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))


currlist= currlist[:24]+["sent"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

49
25


In [54]:
curridx = 45
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = ["have"] + currlist[31:]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

51
21


In [55]:
curridx = 46
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))


currlist = currlist[:20]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

32
20


In [56]:
curridx = 47
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:13]+["stepp*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

29
14


In [57]:
curridx = 48
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:12]+["swam"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

26
13


In [58]:
curridx = 49
currlist = textrows_clean[toolong_idx[curridx]].split(" ")

print(len(currlist))

currlist = currlist[:6]+["throw*"]

textrows_clean2x[toolong_idx[curridx]] = " ".join(currlist)

print(len(currlist))

13
7


In [59]:
textrows_clean2x[2] = textrows_clean2x[2][:56]+" "+ textrows_clean2x[2][56:]


In [60]:
textrows_clean2x[13] = textrows_clean2x[13][:530] + " " + textrows_clean2x[13][530:]

In [61]:
textrows_clean2x[14] = textrows_clean2x[14][1:]

In [62]:
textrows_clean2x[16] = textrows_clean2x[16][:653] + " " + textrows_clean2x[16][653:]

In [63]:
textrows_clean2x[17] = textrows_clean2x[17][:464] + " "+  textrows_clean2x[17][464:]

In [64]:
textrows_clean2x[18] = textrows_clean2x[18][:330] + " " + textrows_clean2x[18][330:]

In [65]:
textrows_clean2x[25]=textrows_clean2x[25][1:]

In [66]:
textrows_clean2x[26] = textrows_clean2x[26][:529] + " " + textrows_clean2x[26][529:]

In [67]:
textrows_clean2x[32] = textrows_clean2x[32][:170]+" "+textrows_clean2x[32][170:]

In [68]:
textrows_clean2x[36] = textrows_clean2x[36][:596]+" "+textrows_clean2x[36][596:]

In [69]:
textrows_clean2x[38] = textrows_clean2x[38][:663]+"m "+textrows_clean2x[38][663:]

In [70]:
textrows_clean2x[46] = textrows_clean2x[46][:381]+" "+textrows_clean2x[46][381:]

In [71]:
textrows_clean2x[47] = textrows_clean2x[47][:368]+" "+textrows_clean2x[47][368:]

In [72]:
textrows_clean2x[48] = textrows_clean2x[48][:368]+" "+textrows_clean2x[48][368:]

In [73]:
textrows_clean2x[68] = textrows_clean2x[68][:98]+"ip "+textrows_clean2x[68][98:]

In [74]:
textrows_clean2x[72] = textrows_clean2x[72][:391] + " " + textrows_clean2x[72][391:]

In [75]:
textrows_clean2x[78] = textrows_clean2x[78][:280]+" "+textrows_clean2x[78][280:]

In [76]:
textrows_clean2x[80] = textrows_clean2x[80][:517]+" "+textrows_clean2x[80][517:]

In [77]:
textrows_clean2x[88] = textrows_clean2x[88][:276]+"* "+textrows_clean2x[88][276:]

In [78]:
textrows_clean2x[89] = textrows_clean2x[89][:277] + " "+textrows_clean2x[89][277:]

In [79]:
textrows_clean2x[90] = textrows_clean2x[90][:150]+" "+textrows_clean2x[90][150:]

In [80]:
textrows_clean2x[104] = textrows_clean2x[104][:354]+"s week"+textrows_clean2x[104][360:]

In [81]:
textrows_clean2x[126] =textrows_clean2x[126][:172] + ' m' + textrows_clean2x[126][172:]

In [82]:
textrows_clean2x[127]  =textrows_clean2x[127] [:161]+" "+textrows_clean2x[127] [161:]

In [83]:
textrows_clean2x[128] = textrows_clean2x[128][:148]+" " + textrows_clean2x[128][148:]

In [84]:
textrows_clean2x[63] = textrows_clean2x[63][:334]+"" +textrows_clean2x[63][339:]

In [85]:
textrows_clean2x[76] = textrows_clean2x[76][:247]+textrows_clean2x[76][252:]

In [86]:
117
textrows_clean2x[117] = textrows_clean2x[117][9:]

In [87]:

textrows_clean2x[-1] = 'went'

textrows_clean2x[-4] = 'walked'

textrows_clean2x[-7] = 'trips'

textrows_clean2x[-10] = 'trip'

textrows_clean2x[-12] = "transport*"

for i in range(len(textrows_clean2x)):
    print(textrows_clean2x[i].split(" "))
    print("")

['a', "he's", 'since', 'anybod*', 'hed', 'i', 'lets', 'thee', 'he', 'their*', 'anybod*', 'a', 'accepted', 'hasnt', 'slept', 'aint', 'accepted', 'admit', "isn't", 'couldve', 'about', 'about', 'also', 'aint', 'all', 'billion*', 'arse', 'acquainta*', 'gives', 'persons', 'aunt*', 'acquainta*', 'adult', 'abandon*', 'damn*', 'fume*', 'kindn*', 'privileg*', 'supporting', 'accept', 'freed*', 'partie*', 'abandon*', 'enrag*', 'maddening', 'snob*', 'afraid', 'abuse*', 'jealous*', 'abandon*', 'abandon*', 'conflict*', 'harness*', 'needed', "should've", 'accept', 'informs', 'activat*', 'besides', 'allot', 'absolute', 'abandon*', 'add', 'but', 'acid*', 'purpl*', 'beaut*', 'audibl*', 'brush*', 'abdomen*', 'disease*', 'intox*', 'scab*', 'abdomen*', 'knuckle*', 'abortion*', 'itch*', 'abortion*', 'alcohol*', 'above', 'constantly', 'goes', 'october', 'spring', 'act', 'above', 'meter*', 'abrupt*', 'like', 'absent*', 'feedback', 'profs', 'abilit*', 'master', 'actor*', 'martini*', 'address', 'account*', 'jac

In [88]:

for j in range(textrows_clean_len):
    
    size = len(textrows_clean2x[j].split(" "))
    if size > basesize:
        print("len longer than max at j = " + str(j) + " with size " + str(size))
        print("")



In [89]:
flag_matrix = np.zeros((len(textrows_clean2x), len(textrows_clean2x[0].split(" "))))*np.nan

#cat1
for i in range(153+1):
    flag_matrix[i,0]=1
    flag_matrix[i,1]=1
    flag_matrix[i,2]=1
    
flag_matrix[153+1,0]=1
flag_matrix[153+1,1]=1

#cat2
for i in range(115+1):
    flag_matrix[i,3]=1
    
    
#cat3
for i in range(69+1):
    flag_matrix[i,4]=1
    
    
#cat4
for i in range(11+1):
    flag_matrix[i,5]=1 
    
    
#cat5
for i in range(11+1):
    flag_matrix[i,6]=1  
    
    
#cat6
for i in range(19+1):
    flag_matrix[i,7]=1 
    
    
#cat7
for i in range(16+1):
    flag_matrix[i,8]=1  
    
    
#cat8
for i in range(9+1):
    flag_matrix[i,9]=1 
    
    
#cat9
for i in range(45+1):
    flag_matrix[i,10]=1 

    
#cat10
for i in range(3+1):
    flag_matrix[i,11]=1
    
#cat11
for i in range(126+1):
    flag_matrix[i,12]=1 
    flag_matrix[i,13]=1 
    flag_matrix[i,14]=1 

flag_matrix[126+1,12]=1
flag_matrix[126+1,13]=1


#cat12
for i in range(143+1):
    flag_matrix[i,15]=1 

    
#cat13
for i in range(144+1):
    flag_matrix[i,16]=1 
    
    
#cat14
for i in range(84+1):
    flag_matrix[i,17]=1 
    flag_matrix[i,18]=1 

flag_matrix[84+1,17]=1   


#cat 15
for i in range(47+1):
    flag_matrix[i,19]=1 
    
    
# cat 16
for i in range(68+1):
    flag_matrix[i,20]=1 
    

# cat 17
for i in range(59+1):
    flag_matrix[i,21]=1
    
    
# cat 18
for i in range(27+1):
    flag_matrix[i,22]=1
    

# cat 19
for i in range(56+1):
    flag_matrix[i,23]=1
    
    
# cat 20
for i in range(88+1):
    flag_matrix[i,24]=1
    
    
# cat 21
for i in range(33+1):
    flag_matrix[i,25]=1
    
    
# cat 22
for i in range(52+1):
    flag_matrix[i,26]=1
    
    
    
# cat 121
for i in range(150+1):
    flag_matrix[i,27]=1
    flag_matrix[i,28]=1
    flag_matrix[i,29]=1

flag_matrix[150+1,27]=1
flag_matrix[150+1,28]=1   
flag_matrix[150+1+1,28]=1  



# cat 122
for i in range(64+1):
    flag_matrix[i,30]=1
    
    
# cat 123
for i in range(36+1):
    flag_matrix[i,31]=1
    
    
# cat 124
for i in range(60+1):
    flag_matrix[i,32]=1
    

# cat 125
for i in range(149+1):
    flag_matrix[i,33]=1
    flag_matrix[i,34]=1
    flag_matrix[i,35]=1
    flag_matrix[i,36]=1
    flag_matrix[i,37]=1
    flag_matrix[i,38]=1
    
for i in range(152+1):
    flag_matrix[i,33]=1
    flag_matrix[i,34]=1
    flag_matrix[i,35]=1
    flag_matrix[i,36]=1
    flag_matrix[i,37]=1

flag_matrix[152+1,35]=1
flag_matrix[152+1,36]=1



# cat 126
for i in range(133+1):
    flag_matrix[i,39]=1
    flag_matrix[i,40]=1
    flag_matrix[i,41]=1
    
flag_matrix[133+1,39]=1
flag_matrix[133+1,40]=1
flag_matrix[133+2,39]=1
flag_matrix[133+2,40]=1
flag_matrix[133+3,40]=1
flag_matrix[133+4,40]=1




# cat 127
for i in range(123+1):
    flag_matrix[i,42]=1
    flag_matrix[i,43]=1
    flag_matrix[i,44]=1
    flag_matrix[i,45]=1

flag_matrix[123+1,42]=1
flag_matrix[123+1,43]=1
flag_matrix[123+1,44]=1


# cat 128
for i in range(90+1):
    flag_matrix[i,46]=1
    
    
# cat 129
for i in range(91+1):
    flag_matrix[i,47]=1
    flag_matrix[i,48]=1
    
    
# cat 130
for i in range(100+1):
    flag_matrix[i,49]=1
    
    
# cat 131
for i in range(145+1):
    flag_matrix[i,50]=1
    flag_matrix[i,51]=1
    flag_matrix[i,52]=1
    flag_matrix[i,53]=1
    flag_matrix[i,54]=1

flag_matrix[145+1,52]=1


# cat 132
for i in range(96+1):
    flag_matrix[i,55]=1
    flag_matrix[i,56]=1
    
flag_matrix[96+1,55]=1


# cat 133
for i in range(107+1):
    flag_matrix[i,57]=1
    
    
# cat 134
for i in range(75+1):
    flag_matrix[i,58]=1
    
    
# cat 135
for i in range(155+1):
    flag_matrix[i,59]=1
    
    
    
# cat 136
for i in range(82+1):
    flag_matrix[i,60]=1
    
    
# cat 137
for i in range(110+1):
    flag_matrix[i,61]=1
    
    
# cat 138
for i in range(17+1):
    flag_matrix[i,62]=1
    
    
# cat 139
for i in range(16+1):
    flag_matrix[i,63]=1
    
    
 # cat 140
for i in range(135+1):
    flag_matrix[i,64]=1
    flag_matrix[i,65]=1
    
flag_matrix[135+1,64]=1


# cat 141
for i in range(71+1):
    flag_matrix[i,66]=1 
    
    
# cat 142
for i in range(50+1):
    flag_matrix[i,67]=1    
    
    
# cat 143
for i in range(74+1):
    flag_matrix[i,68]=1  
    
    
# cat 146
for i in range(140+1):
    flag_matrix[i,69]=1
    flag_matrix[i,70]=1
    flag_matrix[i,71]=1
    flag_matrix[i,72]=1
    
flag_matrix[140+1,69]=1
flag_matrix[140+1,70]=1
flag_matrix[140+1,71]=1



# cat 147
for i in range(89+1):
    flag_matrix[i,73]=1  
    flag_matrix[i,74]=1 
  

# cat 148
for i in range(117+1):
    flag_matrix[i,75]=1  
    flag_matrix[i,76]=1 
    
    
# cat 149
for i in range(94+1):
    flag_matrix[i,77]=1  

    
# cat 150
for i in range(110+1):
    flag_matrix[i,78]=1  

    
# cat 250
for i in range(125+1):
    flag_matrix[i,79]=1  
    flag_matrix[i,80]=1  
    flag_matrix[i,81]=1  
    flag_matrix[i,82]=1  
    flag_matrix[i,83]=1  

for i in range(125+1+2):
    flag_matrix[i,79]=1  
    flag_matrix[i,80]=1  
    flag_matrix[i,81]=1  
    flag_matrix[i,82]=1  

    
# cat 251
for i in range(167+1):
    flag_matrix[i,84]=1 
    
    
# cat 252
for i in range(109+1):
    flag_matrix[i,85]=1 
    flag_matrix[i,86]=1
    
    
# cat 253
for i in range(119+1):
    flag_matrix[i,87]=1 
    flag_matrix[i,88]=1
    
    
    
# cat 354
for i in range(108+1):
    flag_matrix[i,89]=1 
    flag_matrix[i,90]=1
    flag_matrix[i,91]=1
    
    
# cat 355
for i in range(92+1):
    flag_matrix[i,92]=1 
    flag_matrix[i,93]=1
    
    
# cat 356
for i in range(113+1):
    flag_matrix[i,94]=1 
    flag_matrix[i,95]=1
    
flag_matrix[113+1,94]=1


# cat 357
for i in range(92+1):
    flag_matrix[i,96]=1 

    
# cat 358
for i in range(85+1):
    flag_matrix[i,97]=1 
    flag_matrix[i,98]=1 
    
flag_matrix[85+1,97]=1 



# cat 359
for i in range(78+1):
    flag_matrix[i,99]=1 
    flag_matrix[i,100]=1 
    
flag_matrix[78+1,99]=1 


# cat 360
for i in range(61+1):
    flag_matrix[i,101]=1 
    
    

# cat 462
for i in range(29+1):
    flag_matrix[i,102]=1 
    
    

# cat 463
for i in range(7+1):
    flag_matrix[i,103]=1 
    
    
# cat 464
for i in range(8+1):
    flag_matrix[i,104]=1 

In [90]:
len(flag_matrix[:][-1])


textrows_clean2x[curr_rowidx].split(" ")

NameError: name 'curr_rowidx' is not defined

In [91]:
words_matrix = np.empty(flag_matrix.shape, dtype=object)

curr_rowidx = 85
curr_row_flags = flag_matrix[curr_rowidx]
curr_not_nulls_idx = [i for i,x in enumerate(curr_row_flags) if x==x]
curr_text_row = textrows_clean2x[curr_rowidx].split(" ")

print(len(curr_not_nulls_idx))
print(len(curr_text_row))

textrows_clean2x[curr_rowidx].split(" ")

73
72


['do',
 'none',
 'us',
 "we're",
 'ended',
 "must've",
 "weren't",
 "she'd",
 'saw',
 'various',
 'coworker*',
 'male',
 'they',
 'bother*',
 'enrag*',
 'hesita*',
 'neurotic*',
 'sighed',
 'unwelcom*',
 'definitely',
 'joking',
 'talent*',
 'destroy*',
 'inadequa*',
 'revenge*',
 'unsuccessful',
 'uneas*',
 'hellish',
 'war',
 'suffering',
 'binding',
 'everybod*',
 'known',
 'rearrang*',
 'unknow*',
 'ideas',
 'unaccept*',
 'purpose*',
 'most',
 'safe*',
 'harmon*',
 'stink*',
 'checkup*',
 'hair*',
 'pap',
 'toe',
 'itch*',
 'waist*',
 'gyno*',
 'testosterone*',
 'stud',
 'skinny*',
 'brink',
 'fastest',
 'metre*',
 'section',
 'usually',
 'going',
 'inside',
 'town',
 'followup*',
 'tempora*',
 'dorm*',
 'paper*',
 'type*',
 'lesson*',
 'wins',
 'hangin*',
 'team*',
 'studio',
 'invest*',
 'yuan']

In [92]:
words_matrix = np.empty(flag_matrix.shape, dtype=object)


for curr_rowidx in range(len(flag_matrix)):

    curr_row_flags = flag_matrix[curr_rowidx]

    curr_not_nulls_idx = [i for i,x in enumerate(curr_row_flags) if x==x]

    curr_text_row = textrows_clean2x[curr_rowidx].split(" ")


    if len(curr_not_nulls_idx)!=len(curr_text_row):
        print("error at idx " + str(curr_rowidx))
    else:

        for k in range(len(curr_not_nulls_idx)):
            idx = curr_not_nulls_idx[k]
            words_matrix[curr_rowidx, idx] = curr_text_row[k]

error at idx 84
error at idx 85
error at idx 95


In [93]:
textrows_clean2x[84].split(" ")

['difference*',
 'nobod*',
 'upon',
 "we'll",
 'emailed',
 'mustve',
 "we're",
 "shan't",
 'sat',
 'isnt',
 'unique',
 'cousin*',
 'mails',
 'themselves',
 'boring',
 'enjoy*',
 'hero*',
 'nervous*',
 'sigh',
 'unsure*',
 'definite',
 'joke*',
 'sweets',
 'despis*',
 'impolite*',
 'restless*',
 'unsavo*',
 'uncontrol*',
 'hell',
 'violent*',
 'sufferer*',
 'betting',
 'every',
 'knowledg*',
 'really',
 'undoubt*',
 'idea',
 'thoughts',
 'provok*',
 "might've",
 'rigid*',
 'harde*',
 'stench*',
 'cardio*',
 'gyno*',
 'palms',
 'titty',
 'intestin*',
 'vein*',
 'gynecolog**',
 'tendoni*',
 'std*',
 'skinni*',
 'brings',
 'faster',
 'meter*',
 'seconds',
 'usual',
 'goes',
 'inner*',
 'toward*',
 'firsts',
 'synch*',
 'doctor*',
 'overworked',
 'tutor*',
 'lead*',
 'winn*',
 'gym*',
 'tanning',
 'stove*',
 'insurance',
 'yen']

In [94]:
liwc_categories

['Funct',
 'Pronoun',
 'Ppron',
 'I',
 'We',
 'You',
 'SheHe',
 'They',
 'Ipron',
 'Article',
 'Verbs',
 'AuxVb',
 'Past',
 'Present',
 'Future',
 'Adverbs',
 'Prep',
 'Conj',
 'Negate',
 'Quant',
 'Numbers',
 'Swear',
 'Social',
 'Family',
 'Friends',
 'Humans',
 'Affect',
 'Posemo',
 'Negemo',
 'Anx',
 'Anger',
 'Sad',
 'Insight',
 'Cause',
 'Discrep',
 'Tentat',
 'Certain',
 'Inhib',
 'Incl',
 'Excl',
 'Percept',
 'See',
 'Hear',
 'Feel',
 'Bio',
 'Body',
 'Health',
 'Sexual',
 'Ingest',
 'Relativ',
 'Motion',
 'Space',
 'Time',
 'Work',
 'Achiev',
 'Leisure',
 'Home',
 'Money',
 'Relig',
 'Death',
 'Assent',
 'Nonflu',
 'Filler']

In [95]:
liwc_categories[0]

'Funct'

In [96]:
liwc_idx_dict = {
    'Funct': [0,1,2],
 'Pronoun':[3],
 'Ppron':[4],
 'I':[5],
 'We':[6],
 'You':[7],
 'SheHe':[8],
 'They':[9],
 'Ipron':[10],
 'Article':[11],
 'Verbs':[12,13,14],
 'AuxVb':[15],
 'Past':[16],
 'Present':[17,18],
 'Future':[19],
 'Adverbs':[20],
 'Prep':[21],
 'Conj':[22],
 'Negate':[23],
 'Quant':[24],
 'Numbers':[25],
 'Swear':[26],
 'Social':[27,28,29],
 'Family':[30],
 'Friends':[31],
 'Humans':[32],
 'Affect':[33,34,35,36,37,38],
 'Posemo':[39,40,41],
 'Negemo':[42,43,44,45],
 'Anx':[46],
 'Anger':[47,48],
 'Sad':[49],
 'CogMech':[50,51,52,53,54],
 'Insight':[55,56],
 'Cause':[57],
 'Discrep':[58],
 'Tentat':[59],
 'Certain':[60],
 'Inhib':[61],
 'Incl':[62],
 'Excl':[63],
 'Percept':[64,65],
 'See':[66],
 'Hear':[67],
 'Feel':[68],
 'Bio':[69,70,71,72],
 'Body':[73,74],
 'Health':[75,76],
 'Sexual':[77],
 'Ingest':[78],
 'Relativ':[79,80,81,82,83],
 'Motion':[84],
 'Space':[85,86],
 'Time':[87,88],
 'Work':[89,90,91],
 'Achiev':[92,93],
 'Leisure':[94,95],
 'Home':[96],
 'Money':[97,98],
 'Relig':[99,100],
 'Death':[101],
 'Assent':[102],
 'Nonflu':[103],
 'Filler':[104]
}

In [97]:
liwc_dict = {}

df_words = pd.DataFrame(words_matrix)

for cat in liwc_idx_dict:
    idx_list = liwc_idx_dict[cat]
    
    wordslist = []
    for idx in idx_list:
        wordslist.extend(df_words.iloc[:,idx].dropna().tolist())
        
    liwc_dict[cat] = wordslist

In [98]:
liwc_dict["Death"]

['autops*',
 'alive',
 'bereave*',
 'burial*',
 'buried',
 'bury',
 'casket*',
 'casualt*',
 'cemet*',
 'coffin*',
 'coroner*',
 'corpse*',
 'cremat*',
 'crypt*',
 'dead',
 'death*',
 'decease*',
 'demise',
 'die',
 'died',
 'dies',
 'DOA',
 'drown*',
 'dying',
 'embalm*',
 'epidemic*',
 'execution*',
 'exterminat*',
 'fatal*',
 'funer*',
 'genocid*',
 'ghost*',
 'grave*',
 'grief',
 'griev*',
 'hearse*',
 'holocaust*',
 'homocid*',
 'immortal*',
 'kill*',
 'lethal*',
 'lynch*',
 'manslaughter*',
 'massacre*',
 'mausoleum',
 'morgue*',
 'mortal*',
 'mortician',
 'mourn*',
 'murder*',
 'obit*',
 'od',
 'oded',
 'overdosed',
 'pallbearer*',
 'plague*',
 'reaper*',
 'slaughter*',
 'suicid',
 'tomb*',
 'urn*',
 'war']

In [99]:
train_filename = "train2.tsv"


train_file = open(train_filename, 'rb')
lines = train_file.read()
lines = lines.decode("utf-8")

table = []
for line in lines.strip().split('\n'):
    tmp = line.strip().split('\t')
    
    while len(tmp) < 16:
        tmp.append('')
    table.append([tmp[i].strip() for i in range(16)])

In [100]:
'''Column 1: the ID of the statement ([ID].json).
Column 2: the label.
Column 3: the statement.
Column 4: the subject(s).
Column 5: the speaker.
Column 6: the speaker's job title.
Column 7: the state info.
Column 8: the party affiliation.
Column 9-13: the total credit history count, including the current statement.
9: barely true counts.
10: false counts.
11: half true counts.
12: mostly true counts.
13: pants on fire counts.
Column 14: the context (venue / location of the speech or statement).

Column 15 extra: justification  '''

"Column 1: the ID of the statement ([ID].json).\nColumn 2: the label.\nColumn 3: the statement.\nColumn 4: the subject(s).\nColumn 5: the speaker.\nColumn 6: the speaker's job title.\nColumn 7: the state info.\nColumn 8: the party affiliation.\nColumn 9-13: the total credit history count, including the current statement.\n9: barely true counts.\n10: false counts.\n11: half true counts.\n12: mostly true counts.\n13: pants on fire counts.\nColumn 14: the context (venue / location of the speech or statement).\n\nColumn 15 extra: justification  "

In [101]:
df_table = pd.DataFrame(table)
df_table.columns = ["id","json_ID","label","statement","subjects","speaker","speaker_title","state","party",
                   "barely_true_counts","false_counts","half_true_counts","mostly_true_counts","pants_on_fire_counts",
                   "context","justification"]

df_table.head()

,id,json_ID,label,statement,subjects,speaker,speaker_title,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"""Surovell said the decline of coal """"started w..."
2,2,324.json,mostly-true,"""Hillary Clinton agrees with John McCain """"by ...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"""Obama said he would have voted against the am..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,19.0,3.0,5.0,44.0,a news release,"""The release may have a point that Mikulskis c..."
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"""Crist said that the economic """"turnaround sta..."


In [103]:
onestatement = df_table.statement[0]

onestatement = "Anna was about twentyfour when she hits the third trimester of pregnancy"

print(onestatement)


onestatement.split()

Anna was about twentyfour when she hits the third trimester of pregnancy


['Anna',
 'was',
 'about',
 'twentyfour',
 'when',
 'she',
 'hits',
 'the',
 'third',
 'trimester',
 'of',
 'pregnancy']

In [104]:
onestatement

'Anna was about twentyfour when she hits the third trimester of pregnancy'

In [105]:
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer



#tokens = re.findall("all", onestatement)

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(onestatement)
tokens = [token.lower() for token in tokens]
tokens

['anna',
 'was',
 'about',
 'twentyfour',
 'when',
 'she',
 'hits',
 'the',
 'third',
 'trimester',
 'of',
 'pregnancy']

In [227]:
liwc_dict_inv = {}
for key in liwc_dict:
    for word in liwc_dict[key]:
        if word in liwc_dict_inv:
            liwc_dict_inv[word].extend([key])
        else:
            liwc_dict_inv[word]=[key]

In [228]:
liwc_dict_inv["when"]

['Funct', 'Adverbs', 'Conj', 'Relativ', 'Time']

In [248]:
import fnmatch


def makeMatchList(sentence):
    
    idx = sentence[0]
    sentence = sentence[1]
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    tokens = [token.lower() for token in tokens]
    
    matchdict_words = {}
    matchdict_counts = {}
    for token in tokens:
        for key in liwc_dict:
            for wordindict in liwc_dict[key]:
                if fnmatch.fnmatch(token, wordindict):
                    if key in matchdict_words:
                        matchdict_words[key] = matchdict_words[key]+[token]
                        matchdict_counts[key]+=1
                    else:
                        matchdict_words[key] = [token]
                        matchdict_counts[key]=1
                        
    matchdict_counts["totcount"] = len(tokens)  
    matchdict_counts["sentence_idx"] = idx
    return matchdict_counts



def makeMatchList_inv(sentence):

    idx = sentence[0]
    sentence = sentence[1]
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    tokens = [token.lower() for token in tokens]

    matchdict_words = {}

    for token in tokens:
        for wordindict in liwc_dict_inv:
            if fnmatch.fnmatch(token, wordindict): 
                if token in matchdict_words:
                    matchdict_words[token] = matchdict_words[token].append(liwc_dict_inv[wordindict])
                else:
                    matchdict_words[token] = liwc_dict_inv[wordindict]
                    
            continue
                
    #matchdict_counts["totcount"] = len(tokens)  
    #matchdict_counts["sentence_idx"] = idx
    return matchdict_words 



def makeMatchList_inv2(sentence):

    liwc_dict_inv_tmp = liwc_dict_inv.copy()
    idx = sentence[0]
    sentence = sentence[1]
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    tokens = [token.lower() for token in tokens]
    len_sentence = len(tokens)

    matchdict_words = {}

    for token in tokens:
        for wordindict in liwc_dict_inv:
            
            if "*" in wordindict:
                if token.startswith(wordindict.split("*")[0]):
                    
                    if token in matchdict_words:
                        matchdict_words[token] = matchdict_words[token]+liwc_dict_inv_tmp[wordindict]
                    else:
                        matchdict_words[token] = liwc_dict_inv_tmp[wordindict]
                    
                else:
                    continue
                    
            elif len(token)!=len(wordindict):
                continue
            
            elif token==wordindict:
                if token in matchdict_words:
                    matchdict_words[token] = matchdict_words[token]+liwc_dict_inv_tmp[wordindict]
                else:
                    matchdict_words[token] = liwc_dict_inv_tmp[wordindict]
                

    liwc_counts = {}
    for key in matchdict_words:
        for word in matchdict_words[key]:
            if word in liwc_counts:
                liwc_counts[word]+=1/len(tokens)
            else:
                liwc_counts[word]=1/len(tokens) 
    
    return (idx, liwc_counts) 



In [241]:
liwc_counts = {}
for key in matchdict_words:
    for word in matchdict_words[key]:
        if word in liwc_counts:
            liwc_counts[word]+=1/len(tokens)
        else:
            liwc_counts[word]=1/len(tokens) 

NameError: name 'matchdict_words' is not defined

In [242]:
liwc_dict_inv["when"]

['Funct', 'Adverbs', 'Conj', 'Relativ', 'Time']

In [243]:
onestatement

anotherstatement = 'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.'

In [244]:
import time
start = time.time()
print("Time elapsed on working...")
matchdict_counts = makeMatchList_inv2([0,anotherstatement])
end = time.time()
print("Time consumed in working: ",end - start)


matchdict_counts

Time elapsed on working...
when
when
did
did
the
the
decline
of
of
coal
start
start
it
it
started
started
when
when
natural
gas
took
took
off
off
that
that
started
started
to
to
begin
begin
in
in
president
george
w
bushs
administration
{'when': ['Funct', 'Adverbs', 'Conj', 'Relativ', 'Time', 'Funct', 'Adverbs', 'Conj', 'Relativ', 'Time'], 'did': ['Funct', 'Verbs', 'AuxVb', 'Past'], 'the': ['Funct', 'Article'], 'of': ['Funct', 'Prep'], 'start': ['Verbs', 'Present', 'Relativ', 'Time'], 'it': ['Funct', 'Pronoun', 'Ipron'], 'started': ['Verbs', 'Past', 'Relativ', 'Time', 'Verbs', 'Past', 'Relativ', 'Time'], 'took': ['Verbs', 'Past', 'Relativ', 'Motion'], 'off': ['Funct', 'Prep', 'Relativ', 'Space'], 'that': ['Funct', 'Pronoun', 'Ipron'], 'to': ['Funct', 'Prep'], 'begin': ['Verbs', 'Present', 'Relativ', 'Time'], 'in': ['Funct', 'Prep', 'Relativ', 'Space'], 'president': ['Work', 'Achiev'], 'administration': ['Work']}
Time consumed in working:  0.06966400146484375


(0,
 {'Funct': 0.4166666666666667,
  'Adverbs': 0.08333333333333333,
  'Conj': 0.08333333333333333,
  'Relativ': 0.375,
  'Time': 0.24999999999999997,
  'Verbs': 0.24999999999999997,
  'AuxVb': 0.041666666666666664,
  'Past': 0.16666666666666666,
  'Article': 0.041666666666666664,
  'Prep': 0.16666666666666666,
  'Present': 0.08333333333333333,
  'Pronoun': 0.08333333333333333,
  'Ipron': 0.08333333333333333,
  'Motion': 0.041666666666666664,
  'Space': 0.08333333333333333,
  'Work': 0.08333333333333333,
  'Achiev': 0.041666666666666664})

In [245]:
df_dictcounts = df_table.statement.iloc[:2].apply(lambda x: makeMatchList(x))

In [155]:
df_dictcounts

0    {'Funct': 1, 'Article': 1, 'totcount': 1, 'sen...
1                 {'totcount': 1, 'sentence_idx': 'W'}
Name: statement, dtype: object

In [156]:
df_split = np.array_split(df_table.statement.iloc[:5],4)

df_split[0]

0    Says the Annies List political group supports ...
1    When did the decline of coal start? It started...
Name: statement, dtype: object

In [157]:
df_table.statement.iloc[2]

'"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran."""'

In [256]:
statements = []

for j in range(len(df_table)):
    statements.append([(j,df_table.statement.iloc[j])])
    


In [257]:
allres = []
from multiprocessing import Pool

with Pool(processes = 4) as p:
    allres = p.starmap(makeMatchList_inv2, statements)

In [267]:
allres[212][1]

{'Funct': 0.5384615384615385,
 'Pronoun': 0.15384615384615385,
 'Ppron': 0.07692307692307693,
 'You': 0.07692307692307693,
 'Social': 0.15384615384615385,
 'Verbs': 0.23076923076923078,
 'Past': 0.15384615384615385,
 'Percept': 0.07692307692307693,
 'Hear': 0.07692307692307693,
 'Ipron': 0.07692307692307693,
 'AuxVb': 0.15384615384615385,
 'Negate': 0.07692307692307693,
 'CogMech': 0.07692307692307693,
 'Excl': 0.07692307692307693,
 'Affect': 0.07692307692307693,
 'Posemo': 0.07692307692307693,
 'Prep': 0.15384615384615385,
 'Relativ': 0.07692307692307693,
 'Space': 0.07692307692307693}

In [266]:


values = allres[212][1].values()

values

dict_values([0.5384615384615385, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693])

In [2940]:
df_table.index.to_series().map(df_dictcounts)


0        {'Social': 2, 'Percept': 1, 'Hear': 1, 'Funct'...
1        {'Funct': 10, 'Adverbs': 2, 'Conj': 2, 'Relati...
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
10264                                                  NaN
10265                                                  NaN
10266                                                  NaN
10267                                                  NaN
10268                                                  NaN
Length: 10269, dtype: object

In [2652]:
df_table.query("speaker == 'barack-obama'  ").head()

,id,json_ID,label,statement,subjects,speaker,speaker_title,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
2,2,324.json,mostly-true,"""Hillary Clinton agrees with John McCain """"by ...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"""Obama said he would have voted against the am..."
7,7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa.","However, it was not that bill, but another one..."
16,16,620.json,true,McCain opposed a requirement that the governme...,federal-budget,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a radio ad,"""I firmly object to all 'buy America' restrict..."
74,74,10039.json,mostly-true,The cost of health care is now the single-bigg...,"deficit,federal-budget,health-care",barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a speech at Northwestern University,"""Obama saidthe cost of """"health care is now th..."
105,105,338.json,true,Sen. Obama has always had a 100 percent procho...,abortion,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,an e-mail message sent to voters before Super ...,"""The Planned Parenthood Action Fund similarly ..."


### Speakers?

In [8]:
df_table.speaker.unique().shape

(2930,)

In [3201]:
df_table.groupby("speaker").id.count().sort_values(ascending = False).iloc[:300].sum()

6341

In [687]:
df_table.query("speaker == 'allan-fung'  ")

,id,json_ID,label,statement,subjects,speaker,speaker_title,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
1253,1253,9421.json,pants-fire,Ken Block supports Obamacare.,"economy,health-care,message-machine-2014",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a TV commercial,"""The very """"Newsmakers"""" appearance the Fung c..."
1959,1947,10051.json,false,"In the 38 Studios deal, Wall Street investors ...","bankruptcy,debt,economy,history,job-accomplish...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a TV commercial,"""Nelson said, """"My state of Florida is now the..."
3971,3958,10172.json,barely-true,Gina Raimondo paid 70 million in taxpayer doll...,"candidates-biography,corporations,message-mach...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a TV commercial,"Whats more, the average person does not view t..."
5220,5207,9562.json,mostly-true,Cranstons 2014-2015 budget funds 100 percent o...,"economy,income,job-accomplishments,labor,pensi...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a letter to taxpayers,"""We believe in the middle. The independent res..."
5827,5814,9082.json,barely-true,"After hiring a campaign manager in 2006, I got...","economy,government-efficiency,jobs",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a radio interview,Oliver said the Keene Police Department cited ...
6953,6932,8527.json,mostly-true,We have created new jobs here in Cranston -- m...,"city-government,corporations,economy,governmen...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a campaign speech,"Still, even if Todd Palin paid more for his sn..."
8294,8265,9810.json,false,Nazi imagery [was used] by the Block for Gover...,"elections,history,human-rights,campaign-advert...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a news release,"""During the debate Ryan said that Obama was in..."
9829,9800,10129.json,true,Governor Carcieri has not been campaigning wit...,"candidates-biography,debates,campaign-advertising",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a Providence Journal-WPRI-12 debate,The number of flights has dropped from 240 to ...


In [696]:
import wikipedia

print(wikipedia.search("nancy-pelosi "))

['Nancy Pelosi', 'Alexandra Pelosi', 'Christine Pelosi', 'Paul Pelosi', 'Speaker of the United States House of Representatives', 'Susan Page', 'Jim Cramer', 'Ron Pelosi', 'Nancy Mace', 'Steny Hoyer']


In [694]:
print(wikipedia.suggest("nancy pelosi"))

None


In [692]:
print(wikipedia.page(wikipedia.search("nancy-pelosi ")[0]).links)

['100-Hour Plan', '100th United States Congress', '101st United States Congress', '102nd United States Congress', '103rd United States Congress', '104th United States Congress', '105th United States Congress', '106th United States Congress', '107th United States Congress', '108th United States Congress', '109th United States Congress', '110th United States Congress', '111th United States Congress', '112th Congress', '112th United States Congress', '113th United States Congress', '114th United States Congress', '115th United States Congress', '116th United States Congress', '117th United States Congress', '14th Dalai Lama', '1828 United States presidential election', '1832 Democratic National Convention', '1835 Democratic National Convention', '1840 Democratic National Convention', '1844 Democratic National Convention', '1848 Democratic National Convention', '1852 Democratic National Convention', '1856 Democratic National Convention', '1860 Democratic National Conventions', '1864 Democr

In [43]:
print(wikipedia.page('Nancy Pelosi').categories)

['1940 births', '20th-century American politicians', '20th-century American women politicians', '21st-century American politicians', '21st-century American women politicians', 'All BLP articles lacking sources', 'All Wikipedia articles written in American English', 'American people of Italian descent', 'American women activists', 'Articles containing video clips', 'Articles with Curlie links', 'Articles with short description', 'BLP articles lacking sources from January 2021', 'CS1 maint: extra text: authors list', 'California Democratic Party chairs', 'California Democrats', 'Catholics from California', 'Catholics from Maryland', 'Democratic Party members of the United States House of Representatives', 'Female members of the United States House of Representatives', 'Good articles', 'Italian-American culture in Baltimore', 'Italian-American culture in California', 'LGBT rights activists from the United States', 'Living people', 'Members of the United States House of Representatives fro

In [40]:
print(wikipedia.page('Nancy Pelosi').content)

Nancy Patricia Pelosi (; née D'Alesandro; born March 26, 1940) is an American politician serving as Speaker of the United States House of Representatives since 2019, and previously from 2007 to 2011. She has served as a U.S. representative from California since 1987. A member of the Democratic Party, Pelosi is the only woman in U.S. history to serve as speaker. She is second in the presidential line of succession, after vice president Kamala Harris. She is the dean of California's Congressional delegation.
Pelosi was first elected to Congress in 1987, following her father, Thomas D'Alesandro Jr., who served as a U.S. representative from Maryland and Mayor of Baltimore, into politics. She is the dean of California's congressional delegation, having begun her 18th term in 2021. Pelosi represents California's 12th congressional district, which comprises four-fifths of the city and county of San Francisco. She initially represented the 5th district (1987–1993), and then, when district boun